In [1]:
import os

from datasets import load_dataset, DatasetDict, Dataset, load_from_disk
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel, MambaConfig
import torch
from transformers import AutoConfig, AutoTokenizer, DataCollatorForLanguageModeling, PretrainedConfig, MambaForCausalLM

In [2]:
CONTEXT_LENGTH = 4096
DATASET_PATH = './data/tokenized_dataset_130g_4k_v2/'
TOKENIZER_PATH = './tokenizer/tokenizer_v4/'
MODEL_PATH = './model/komamba_deepspeed_130m_130g_2k_2k'
OUTPUT_PATH = './output_v2'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

In [4]:
tokenized_datasets = load_from_disk(DATASET_PATH)

In [5]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4772510
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 531549
    })
})

model_path = 'state-spaces/mamba-370m'

xmodel = MambaLMHeadModel.from_pretrained(model_path, dtype=torch.float16)

conf = xmodel.config

conf


conf = MambaConfig(d_model=768,
                   n_layer=24,
                   vocab_size=51200,
                   ssm_cfg={},
                   rms_norm=True,
                   residual_in_fp32=True,
                   fused_add_norm=True,
                   pad_vocab_size_multiple=8)

mamba-370m

MambaConfig(d_model=1024, n_layer=48, vocab_size=50277, ssm_cfg={}, rms_norm=True, residual_in_fp32=True, fused_add_norm=True, pad_vocab_size_multiple=8)

mamba-790m

MambaConfig(d_model=1536, n_layer=48, vocab_size=50277, ssm_cfg={}, rms_norm=True, residual_in_fp32=True, fused_add_norm=True, pad_vocab_size_multiple=8)


mamba-1.4b

MambaConfig(d_model=2048, n_layer=48, vocab_size=50277, ssm_cfg={}, rms_norm=True, residual_in_fp32=True, fused_add_norm=True, pad_vocab_size_multiple=8)


mamba-2.8b

MambaConfig(d_model=2560, n_layer=64, vocab_size=50277, ssm_cfg={}, rms_norm=True, residual_in_fp32=True, fused_add_norm=True, pad_vocab_size_multiple=8)

In [6]:
conf = MambaConfig(d_model=768,
                   n_layer=24,
                   vocab_size=51200,
                   ssm_cfg={},
                   rms_norm=True,
                   residual_in_fp32=True,
                   fused_add_norm=True,
                   pad_vocab_size_multiple=8)

In [7]:
conf

MambaConfig(d_model=768, n_layer=24, vocab_size=51200, ssm_cfg={}, rms_norm=True, residual_in_fp32=True, fused_add_norm=True, pad_vocab_size_multiple=8)

In [8]:
model = MambaLMHeadModel(config=conf)

In [9]:
conf = {"d_model": 768, 
        "n_layer": 24, 
        "vocab_size": 51200, 
        "ssm_cfg": {}, 
        "rms_norm": True, 
        "residual_in_fp32": True, 
        "fused_add_norm": True, 
        "pad_vocab_size_multiple": 8}

model.config = PretrainedConfig(**{**conf})

In [10]:
model

MambaLMHeadModel(
  (backbone): MixerModel(
    (embedding): Embedding(51200, 768)
    (layers): ModuleList(
      (0-23): 24 x Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=768, out_features=3072, bias=False)
          (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
          (act): SiLU()
          (x_proj): Linear(in_features=1536, out_features=80, bias=False)
          (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
          (out_proj): Linear(in_features=1536, out_features=768, bias=False)
        )
        (norm): RMSNorm()
      )
    )
    (norm_f): RMSNorm()
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
model.to(device)
0

NameError: name 'model' is not defined

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

In [10]:
def generate(model, tokenizer, device, prompt, max_length=20):
    """Generate text using given prompt"""

    input_ids = tokenizer(prompt, return_tensors='pt')["input_ids"].to(device=device)
    out = model.generate(input_ids, max_new_tokens=max_length)
    return tokenizer.batch_decode(out)

In [15]:
def generate(model, tokenizer, device, prompt, max_length=20):
    """Generate text using given prompt"""

    inputs = tokenizer(prompt, return_tensors='pt')
    input_ids = inputs.input_ids.to(device=device)
    attn_mask = inputs.attention_mask.to(device=device)

    generate_ids = model.generate(input_ids=input_ids,
                                  max_length=input_ids.shape[1]+max_length,
                                  cg=True,
                                  return_dict_in_generate=True,
                                  output_scores=True,
                                  enable_timing=False,
                                  temperature=1.0,
                                  top_k=1,
                                  top_p=1.0,
                                  repetition_penalty=1.0)
    generate_ids.sequences

    return tokenizer.batch_decode(generate_ids.sequences, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


In [16]:
prompt = "카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을"
out = generate(model, tokenizer, device, prompt)
str(out)

'카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을 행동을'

In [18]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

out = data_collator([tokenized_datasets['train'][i] for i in range(3)])

for key in out:
    print(f"{key}: {out[key].shape}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids: torch.Size([3, 4098])
attention_mask: torch.Size([3, 4098])
labels: torch.Size([3, 4098])


In [19]:
out['input_ids'][0][:20], out['attention_mask'][0][:20], out['labels'][0][:20]

(tensor([  522,   483,   483,  6667,  7665, 10485, 33700, 10486, 34038,  2346,
          6451, 22681,  6187,  6954,  6490, 15479,  3753, 28367,  6259,  1185]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 tensor([  522,   483,   483,  6667,  7665, 10485, 33700, 10486, 34038,  2346,
          6451, 22681,  6187,  6954,  6490, 15479,  3753, 28367,  6259,  1185]))

In [20]:
from transformers import TrainingArguments

batch_size=16
logging_steps=1000

args = TrainingArguments(
    output_dir="kowikimamba",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_steps=logging_steps,
    logging_steps=logging_steps,
    gradient_accumulation_steps=8,
    weight_decay=0.1,
    warmup_steps=logging_steps,
    max_steps=7000,
    #num_train_epochs=10,
    learning_rate=5e-5,
    save_steps=5000,
    fp16=True,
    optim="adamw_torch"
)

In [21]:
from transformers import Trainer

class MambaTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        input_ids = inputs.pop("input_ids")
        lm_logits = model(input_ids).logits

        labels = input_ids.to(lm_logits.device)
        shift_logits = lm_logits[:, :-1, :].contiguous()
        labels = labels[:, 1:].contiguous()

        loss_fct = torch.nn.CrossEntropyLoss()
        lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), labels.view(-1))

        return lm_loss

    def save_model(self, output_dir, _internal_call):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        torch.save(self.model.state_dict(), f"{output_dir}/pytorch_model.bin")
        self.tokenizer.save_pretrained(output_dir)

In [23]:
trainer = MambaTrainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

In [24]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 25.01 GiB. GPU 0 has a total capacty of 39.39 GiB of which 8.60 GiB is free. Including non-PyTorch memory, this process has 30.79 GiB memory in use. Of the allocated memory 28.11 GiB is allocated by PyTorch, and 522.85 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

(1) 790m, 77g, 10k steps, 19.33 hours

In [26]:
model.save_pretrained("./model/komamba_790m_7k_f16_77g", ) 

In [27]:
trainer.save_model("./model/komamba_790m_7k_f16_77g") 

TypeError: save_model() missing 1 required positional argument: '_internal_call'

In [25]:
prompt = """카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을"""
inputs = tokenizer(prompt, return_tensors="pt")







nerate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 취하는 것을 거부했다.'

(1) '카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 할 수 있는 능력을 가진 것으로 생각한다고 말했다.'

(2) '카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 취해야 한다고 주장했다.'

(3) '카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 취해야 한다고 주장했다.'

In [26]:
prompt = """활동은 소설, 극작, 수필, 번역, 라디오 드라마와 영화 시나리오 등 다양하다. 1974년부터 1991년까지 공산당에 입당하고 있으며, 초기에는 막시즘 관점에서"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,python zero_to_fp32.py . pytorch_model.bin
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'활동은 소설, 극작, 수필, 번역, 라디오 드라마와 영화 시나리오 등 다양하다. 1974년부터 1991년까지 공산당에 입당하고 있으며, 초기에는 막시즘 관점에서 활동했다.'

(1) '활동은 소설, 극작, 수필, 번역, 라디오 드라마와 영화 시나리오 등 다양하다. 1974년부터 1991년까지 공산당에 입당하고 있으며, 초기에는 막시즘 관점에서 공산주의의 이념과 이념의 변화를 강조하면서 정치·경제·정치·예술 등 다양한 분야에서 활동했다.'

(2) '활동은 소설, 극작, 수필, 번역, 라디오 드라마와 영화 시나리오 등 다양하다. 1974년부터 1991년까지 공산당에 입당하고 있으며, 초기에는 막시즘 관점에서 사회 개혁 운동을 주도했으며, 특히 제2차 세계 대전 당시 나치 독일의 나치당 지도자였던 아돌프 히틀러에 의해 나치당에 가담하였다.'

(3) '활동은 소설, 극작, 수필, 번역, 라디오 드라마와 영화 시나리오 등 다양하다. 1974년부터 1991년까지 공산당에 입당하고 있으며, 초기에는 막시즘 관점에서만 활동했으나 현재는 중도좌파 성향의 정당이다.'

In [27]:
prompt = """근육이 커지기 위해서는"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'근육이 커지기 위해서는 근육이 필요하다. 근육이 근육을 만들어주는 것은 근육이기 때문이다. 근육이 근육을 만들어주는 것은 근육이 근육을 만들어주는 것과 같다. 근육이 근육을 만들어주는 것은 근육을 만들어주는 것이다. 근육이 근육을 만들어주는 것은 근육을 만들어주는 것이다.'

(1) '근육이 커지기 위해서는, 운동하는 운동 운동이나 운동 운동이나 운동 운동에도 사용된다.'

(2) '근육이 커지기 위해서는, 이 두 가지 중 어느 하나라도 더하면 안 되는 것이다.'

(3) '근육이 커지기 위해서는, 근육이나 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육, 근육'


In [28]:
prompt = """피부의 조직이 괴사된다면"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'피부의 조직이 괴사된다면, 피부는 피부의 조직을 파괴하는 것이 아니라 피부의 조직을 파괴하는 것이다.'

(1) '피부의 조직이 괴사된다면 뇌가 뇌에 침투하여 뇌에 손상을 입히거나 손상되지 않도록 한다.'

(2) '피부의 조직이 괴사된다면 근육과 근육 사이의 근육 관계는 점점 더 악화된다.'

(3) '피부의 조직이 괴사된다면 그 근육은 근육을 자극하여 근육을 자극하게 된다.'

(1) 130m, kowiki, batch_size=48, 10 epochs, 21.35h

(2) 370m, kowiki, batch_size=48, 10000 steps, 28.40h 

(3) 790m, kowiki, batch_size=16, f16, 10000 steps, 28.40h 

(4) 1.3b, kowiki, batch_size=, steps,

(5) 2.8b, kowiki, batch_size=, steps,

In [30]:
from mamba_ssm.utils.hf import load_config_hf, load_state_dict_hf

class MambaLMHeadModel2(MambaLMHeadModel):
    def __init__(
        self,
        config: MambaConfig,
        initializer_cfg=None,
        device=None,
        dtype=None,
    ) -> None:
        super().__init__(config, initializer_cfg, device, dtype)

    @classmethod
    def from_pretrained(cls, pretrained_model_name, device=None, dtype=None, **kwargs):
        config = load_config_hf(pretrained_model_name)
        model = cls(**config, device=device, dtype=dtype, **kwargs)
        model.load_state_dict(load_state_dict_hf(pretrained_model_name, device=device, dtype=dtype))
        return model    

In [6]:
print('Load the saved model\n')
#new_model = MambaLMHeadModel.from_pretrained('./model', dtype=torch.float16)
new_model = MambaForCausalLM.from_pretrained(MODEL_PATH, use_safetensors=True)

Load the saved model



In [7]:
new_model.to(device)

MambaForCausalLM(
  (backbone): MambaModel(
    (embeddings): Embedding(51200, 768)
    (layers): ModuleList(
      (0-23): 24 x MambaBlock(
        (norm): MambaRMSNorm()
        (mixer): MambaMixer(
          (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
          (act): SiLU()
          (in_proj): Linear(in_features=768, out_features=3072, bias=False)
          (x_proj): Linear(in_features=1536, out_features=80, bias=False)
          (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
          (out_proj): Linear(in_features=1536, out_features=768, bias=False)
        )
      )
    )
    (norm_f): MambaRMSNorm()
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [11]:
print('\n====== Check out Saved Model  ======\n')

prompt1 = """카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을"""
out1 = generate(new_model, tokenizer, device, prompt1)


prompt2 = """활동은 소설, 극작, 수필, 번역, 라디오 드라마와 영화 시나리오 등 다양하다. 1974년부터 1991년까지 공산당에 입당하고 있으며, 초기에는 막시즘 관점에서"""
out2 = generate(new_model, tokenizer, device, prompt2)


prompt3 = """근육이 커지기 위해서는"""
out3 = generate(new_model, tokenizer, device, prompt3)



====== Check out Saved Model  ======



In [12]:
out1

['카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 ']

In [13]:
out2

['활동은 소설, 극작, 수필, 번역, 라디오 드라마와 영화 시나리오 등 다양하다. 1974년부터 1991년까지 공산당에 입당하고 있으며, 초기에는 막시즘 관점에서의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 ']

In [14]:
out3

['근육이 커지기 위해서는의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 의 ']

In [62]:
model_path = './model'
model_name = 'komamba'

if not os.path.isdir(os.path.join(model_path, model_name)):
    os.mkdir(os.path.join(model_path, model_name))

In [48]:
!pip install trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 2.0 MB/s eta 0:00:002.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 2.3 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 3.4 MB/s eta 0:00:00


In [49]:
from trl import SFTTrainer

In [59]:
dataset = load_dataset('nsmc')

In [60]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [85]:
train_dataset = dataset["train"].train_test_split(test_size=0.1)

In [86]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label', 'input'],
        num_rows: 135000
    })
    test: Dataset({
        features: ['id', 'document', 'label', 'input'],
        num_rows: 15000
    })
})

In [61]:
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'document': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

In [323]:
int2label = {0: '부정', 1: '긍정'}
label2int = {'부정': 0, '긍정': 1}

In [68]:
prompt = """질문의 내용이 긍정적인지 부정적인지 답변해 (질문): 아 더빙.. 진짜 짜증나네요 목소리 (답):"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'질문의 내용이 긍정적인지 부정적인지 답변해 (질문): 아 더빙.. 진짜 짜증나네요 목소리 (답): "할아버지의 말"'

In [77]:
prompt = """질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 너무재밓었다그래서보는것을추천한다\n (답):"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'질문의 내용이 긍정적인지 부정적인지 답변해 (질문): 너무재었다그래서보는것을추천한다 (답): "직접적으로"'

In [78]:
prompt = """질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해\n (답):"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'질문의 내용이 긍정적인지 부정적인지 답변해 (질문): 울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해 (답): "차라리 안 되니?"'

In [79]:
prompt = """질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해\n (답):"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'질문의 내용이 긍정적인지 부정적인지 답변해 (질문): 울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해 (답): "차라리 안 되니?"'

In [87]:
train_dataset['train']

Dataset({
    features: ['id', 'document', 'label', 'input'],
    num_rows: 135000
})

In [88]:
train_dataset['test']

Dataset({
    features: ['id', 'document', 'label', 'input'],
    num_rows: 15000
})

In [89]:
dataset['test']

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 50000
})

In [166]:
from datasets import DatasetDict
import random

tokenizer.padding_side='left'

prompt_format1 = """질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): %s\n (답):%s"""
prompt_format2 = """다음 질문은 긍정일까요 부정일까요?\n (질문): %s\n (답):%s"""
prompt_format3 = """질문이 긍정적이야? 아니면 부정적이야?\n (질문): %s \n(답):%s"""

prompts = [prompt_format1, prompt_format2, prompt_format3]
def gen_prompt_nsmc(element):
    #prompt_format = prompts[random.randint(0, len(prompts)-1)]
    prompt_format = prompts[0]
    return DatasetDict({'input': prompt_format%(element['document'], int2label[element['label']])})


train_dataset['train'] = train_dataset['train'].map(gen_prompt_nsmc)
train_dataset['test'] = train_dataset['test'].map(gen_prompt_nsmc)
dataset['test'] = dataset['test'].map(gen_prompt_nsmc)

Map:   0%|          | 0/135000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [167]:
train_dataset['train'][0]

{'id': '6049534',
 'document': '다른시각으로서의테무진아쉬움투성이영화는영화라지만역사와크게틀리다',
 'label': 0,
 'input': '질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 다른시각으로서의테무진아쉬움투성이영화는영화라지만역사와크게틀리다\n (답):부정'}

In [168]:
train_dataset['train'].to_pandas()

,id,document,label,input
0,6049534,다른시각으로서의테무진아쉬움투성이영화는영화라지만역사와크게틀리다,0,질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 다른시각으로서의테무진아쉬움...
1,9066833,이건 대체 뭐하자는거지,0,질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 이건 대체 뭐하자는거지\n...
2,6708819,이게 뭐야...액션도 토하겠고...,0,질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 이게 뭐야...액션도 토하...
3,9447159,나만당할수는없지ㅋㅋㅋ꼭보시길,1,질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 나만당할수는없지ㅋㅋㅋ꼭보시...
4,9943250,전당포는 안하시는 우주파괴급 파워 아저씨,1,질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 전당포는 안하시는 우주파괴...
...,...,...,...,...
134995,5555645,마지막 그 한마디의 북받침이란...,1,질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 마지막 그 한마디의 북받침...
134996,8694184,재미없습니다 그리고 신음 소리땜에 당황했네요 아 물런 전 21살 영화가 참 재미없어...,0,질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 재미없습니다 그리고 신음 ...
134997,3359765,이런 영화는 앞으로 만들지마라..배우들도 작품선택좀 잘하고...,0,질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 이런 영화는 앞으로 만들지...
134998,4301777,로버트 패틴슨의 안습인 연기.... 내 달리를 망쳤어 ㅠㅜ,0,질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 로버트 패틴슨의 안습인 연...


In [169]:
def tokenize(element):
    tokenizer.pad_token = tokenizer.eos_token
    outputs = tokenizer(
        element['input'],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=False,
        return_length=True,
        padding='max_length'
    )
    return {"input_ids": outputs["input_ids"]}


context_length=128
tokenized_datasets = train_dataset.map(
    tokenize, batched=True, remove_columns=train_dataset['train'].column_names
)
tokenized_datasets

Map:   0%|          | 0/135000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 135000
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 15000
    })
})

In [170]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [171]:
out = data_collator([tokenized_datasets['train'][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


In [310]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="kowikimamba",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_steps=1000,
    logging_steps=1000,
    gradient_accumulation_steps=8,
    weight_decay=0.1,
    #warmup_steps=logging_steps,
    #max_steps=1000,
    num_train_epochs=3,
    learning_rate=5e-5,
    save_steps=5000,
    fp16=True,
    optim="adamw_torch"
)

trainer = MambaTrainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [311]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=789, training_loss=0.17450283809608683, metrics={'train_runtime': 5428.1994, 'train_samples_per_second': 74.61, 'train_steps_per_second': 0.145, 'total_flos': 0.0, 'train_loss': 0.17450283809608683, 'epoch': 2.99})

In [312]:
prompt = """질문의 내용이 긍정적인지 부정적인지 답변해 (질문): 아 더빙.. 진짜 짜증나네요 목소리 (답):"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'질문의 내용이 긍정적인지 부정적인지 답변해 (질문): 아 더빙.. 진짜 짜증나네요 목소리 (답):부정적으로 모든게 추천하고 싶은 영화..고맙습니다..내가정말좋은일도있고이쁜게있으니까고나왔으면좋겠는데..이런글남긴시작같고.답답해.이태재개막장..개재미없어 (답):부정잡아할때만보면아직도죽는줄..와.. (답):부정잡아님. (답):부정잡종'

In [313]:
prompt = """질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 너무재밓었다그래서보는것을추천한다\n (답):"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'질문의 내용이 긍정적인지 부정적인지 답변해 (질문): 너무재었다그래서보는것을추천한다 (답):긍정적인귀신이란사람들은고양이야.그래도솔직히강윤재너무불쌍하지않음??줄거다줬는데 진심으로~그거알면서도딴남자만나네?얼라들이이거보고잘배우겠네?이러닌깐여자들이여우라는소리듣는거야~졸라재미없어~안봐이제..작가누구야? (답):부정잡종아닌가요'

In [314]:
prompt = """질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해\n (답):"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'질문의 내용이 긍정적인지 부정적인지 답변해 (질문): 울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해 (답):부정 연기도 너무 좋았음. 그리고 개콘감들은 다 안쓰럽게 웃긴건 또 보고싶다는거냐.내가 사람한텐 할일없는사람이이이스토리를 이제 이렇게 잘만들면 이유가 뭘까...;;;; (답):긍정적인나저나 (답):긍정적인나저나 (답):긍정적인**************'

In [315]:
dataset['test'] 

Dataset({
    features: ['id', 'document', 'label', 'input'],
    num_rows: 50000
})

In [316]:
dataset['test'] [123]

{'id': '9501525',
 'document': '훌륭한 작가와 뛰어난 연출진이 만나, 훌륭하고 뛰어난 한 편의 드라마가 만들어졌다.',
 'label': 1,
 'input': '질문의 내용이 긍정적인지 부정적인지 답변해\n (질문): 훌륭한 작가와 뛰어난 연출진이 만나, 훌륭하고 뛰어난 한 편의 드라마가 만들어졌다.\n (답):긍정'}

In [317]:
valid_dataset = dataset['test'].map(
    tokenize, batched=True, remove_columns=['id', 'input', 'document']
)
valid_dataset

Dataset({
    features: ['label', 'input_ids'],
    num_rows: 50000
})

In [318]:
from torch.utils.data import DataLoader

batch_size=16
val_ds = valid_dataset
val_ds.set_format(type='torch')
val_dl = DataLoader(val_ds, batch_size=batch_size)

In [319]:
import re
import torch
from tqdm import tqdm

def acc(pred,label):
    return torch.sum(torch.tensor(pred) == label.squeeze()).item()

In [325]:
model.eval()
val_losses = []
val_acc = 0

for step, batch in enumerate(tqdm(val_dl)):
    label = batch['label']
    input_id= batch['input_ids'].to(device)

    pred = model.generate(input_id, max_length=150)
    decoded_pred = tokenizer.batch_decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    decoded_pred = [re.findall("\(답\):[부정|긍정]*", x)[0][4:6] if re.findall("\(답\):[부정|긍정]*", x) else 'none' for x in decoded_pred]
    decoded_pred = [label2int[x] if x in label2int else -1 for x in decoded_pred]
    val_acc += acc(decoded_pred, label)
    

print("val acc: ", val_acc/len(val_dl.dataset))

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [31:57<00:00,  1.63it/s]

val acc:  0.99988


(1) 790m(10000 steps), 3epochs, prompt#0, 0.99988

In [326]:
val_acc

49994

In [ ]:
0.50346

### KLUE NLI

In [327]:
nli_dataset = load_dataset('klue', 'nli')

In [328]:
nli_dataset

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 24998
    })
    validation: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 3000
    })
})

In [329]:
len(nli_dataset['train'])

24998

In [330]:
len(nli_dataset['validation'])

3000

In [331]:
nli_train_dataset = nli_dataset["train"].train_test_split(test_size=0.1)

In [332]:
nli_train_dataset

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 22498
    })
    test: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 2500
    })
})

In [333]:
nli_train_dataset['train'].features

{'guid': Value(dtype='string', id=None),
 'source': Value(dtype='string', id=None),
 'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)}

In [334]:
nli_dataset['validation'].features

{'guid': Value(dtype='string', id=None),
 'source': Value(dtype='string', id=None),
 'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)}

In [335]:
int2label = {0: '함의', 1: '중립', 2: '모순'}
label2int = {'함의': 0, '중립': 1, '모순':2}

In [336]:
from datasets import DatasetDict
import random

tokenizer.padding_side='left'

prompt_format1 = """다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1): %s\n (문장2): %s\n (답):%s"""
prompt_format2 = """다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1): %s\n (문장2): %s\n (답):%s"""
prompt_format3 = """다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1): %s\n (문장2): %s\n (답):%s"""

prompts = [prompt_format1, prompt_format2, prompt_format3]
def gen_prompt_nli(element):
    #prompt_format = prompts[random.randint(0, len(prompts)-1)]
    prompt_format = prompts[0]
    return DatasetDict({'input': prompt_format%(element['premise'], element['hypothesis'], int2label[element['label']])})


nli_train_dataset['train'] = nli_train_dataset['train'].map(gen_prompt_nli)
nli_train_dataset['test'] = nli_train_dataset['test'].map(gen_prompt_nli)
nli_dataset['validation'] = nli_dataset['validation'].map(gen_prompt_nli)

Map:   0%|          | 0/22498 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [337]:
nli_train_dataset['train'][0]

{'guid': 'klue-nli-v1_train_20786',
 'source': 'airbnb',
 'premise': '집주인이 너무 친절하며 답장도 굉장히 빠릅니다.',
 'hypothesis': '집주인의 답장은 천천히 옵니다.',
 'label': 2,
 'input': '다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1): 집주인이 너무 친절하며 답장도 굉장히 빠릅니다.\n (문장2): 집주인의 답장은 천천히 옵니다.\n (답):모순'}

In [338]:
nli_train_dataset['train'].to_pandas()

,guid,source,premise,hypothesis,label,input
0,klue-nli-v1_train_20786,airbnb,집주인이 너무 친절하며 답장도 굉장히 빠릅니다.,집주인의 답장은 천천히 옵니다.,2,"다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1..."
1,klue-nli-v1_train_17382,wikitree,이외에도 콘크리트 발판을 추가하거나 자갈을 깔 수도 있다.,콘크리트 발판을 더하거나 작은 돌들을 놓을 수도 있다.,0,"다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1..."
2,klue-nli-v1_train_16901,wikitree,이번 협약은 세 기관 간 해외투자유치 협력체계를 구축하고 이를 통한 시너지 창출 및...,이번 협약의 세 기관 모두 투자증권 회사이다.,1,"다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1..."
3,klue-nli-v1_train_05101,NSMC,누구라도 손에 꼽을 사상최고의 걸작만화.,최고의 걸작 만화로 뽑힌 걸작.,1,"다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1..."
4,klue-nli-v1_train_07914,policy,"먼저 중소기업과 소상공인 제품을 중심으로 민간 쇼핑몰, 가치삽시다 플랫폼 등을 통해...",먼저 중소기업과 소상공인 제품을 중심으로 민간 쇼핑몰 등을 통해 오프라인 방식의 판...,2,"다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1..."
...,...,...,...,...,...,...
22493,klue-nli-v1_train_02189,wikitree,공예명장관에서는 송현경 대한민국 명장을 포함한 한경희 광주 공예명장 등의 작품 20...,공예명장관에서 명장들의 작품을 보여줄 예정이다.,0,"다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1..."
22494,klue-nli-v1_train_04802,NSMC,내가 영화론과 예술을이정도로 잊고있었던건가.,많은 것을 잊고 있었다.,1,"다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1..."
22495,klue-nli-v1_train_05668,wikipedia,당시 바그너는 스위스 루체른의 교외에 있는 트리프센에 거주하면서 작곡과 논문을 집필...,바그너는 트리프센에 거주했다.,0,"다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1..."
22496,klue-nli-v1_train_22113,policy,"태안 해수욕장 중 유일하게 입장료가 있는 해수욕장이며 입장료 안에는 주차료, 샤워비...",해수욕장의 입장료는 일인당 15000원 이다.,1,"다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1..."


In [339]:
def tokenize(element):
    tokenizer.pad_token = tokenizer.eos_token
    outputs = tokenizer(
        element['input'],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=False,
        return_length=True,
        padding='max_length'
    )
    return {"input_ids": outputs["input_ids"]}


context_length=128
tokenized_nli_datasets = nli_train_dataset['train'].map(
    tokenize, batched=True, remove_columns=nli_train_dataset['train'].column_names
)
tokenized_nli_datasets

Map:   0%|          | 0/22498 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids'],
    num_rows: 22498
})

In [340]:
tokenized_nli_validation_datasets = nli_dataset['validation'].map(
    tokenize, batched=True, remove_columns=nli_dataset['validation'].column_names
)
tokenized_nli_validation_datasets

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids'],
    num_rows: 3000
})

In [341]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [342]:
out = data_collator([tokenized_datasets['train'][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


In [343]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="kowikimamba",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_steps=1000,
    logging_steps=1000,
    gradient_accumulation_steps=8,
    weight_decay=0.1,
    #warmup_steps=logging_steps,
    #max_steps=1000,
    num_train_epochs=3,
    learning_rate=5e-5,
    save_steps=5000,
    fp16=True,
    optim="adamw_torch"
)

trainer = MambaTrainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_nli_datasets,
    eval_dataset=tokenized_nli_validation_datasets
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [344]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=132, training_loss=0.2015042593984893, metrics={'train_runtime': 907.0973, 'train_samples_per_second': 74.407, 'train_steps_per_second': 0.146, 'total_flos': 0.0, 'train_loss': 0.2015042593984893, 'epoch': 3.0})

In [345]:
prompt = """다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1): 10년전 나를 매료시켰던 묘하고 멋진영화\n (문장2): 10년 전에 나를 실망시켰던 영화\n (답):"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘. (문장1): 10년전 나를 매료시켰던 묘하고 멋진영화 (문장2): 10년 전에 나를 실망시켰던 영화 (답):모순한사랑의 영화. (문장2): 10년 전에 나를 실망시켰던 영화. (답):모순한사랑의 사랑과 가족. (답):모순한사랑의 행복한 인생이라는 느낌은 너무도 먹먹하다. (답):중립적인 사랑을 보여주는 영화. (답):모순하고 있다. (답):중립적인 분위기가 오래도록 살아있다. (답):중립적인 분위기가 가장 잘 드러나는'

In [346]:
prompt = """다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1): 10여 년 만에 출소한 금고 털이범 아버지 정도의 등장으로 아들 봉수와 며느리는 놀란다.\n (문장2): 정도는 감옥에서 모범수였다.\n (답):"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘. (문장1): 10여 년 만에 출소한 금고 털이범 아버지 정도의 등장으로 아들 봉수와 며느리는 놀란다. (문장2): 정도는 감옥에서 모범수였다. (답):중립으로 인한 범죄를 저질렀다. (답):모순희 씨의 개인 취권은 서울 강남구 봉은사로에 위치한 바비레드 강남점에 인앤아웃 버거 팝업스토어가 열렸다. (답):모순하우는 서울 강남의 일드인게 아닐까 한다. (답):모순한모순유순가어이가없어요. (답):모순하얼은중립적 정신은 뭘까? (답):모'

In [347]:
prompt = """다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘.\n (문장1): 10점 만점에 10점을 줘도 부족한 드라마입니다.\n (문장2): 드라마의 내용이 만점입니다.\n (답):"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device=device)
attn_mask = inputs.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100


# Generate
fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1.0,
)

out = fn()
tokenizer.batch_decode(out.sequences.tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'다음 문장1과 문장2의 관계를 함의, 중립, 모순 중 하나로 분류해줘. (문장1): 10점 만점에 10점을 줘도 부족한 드라마입니다. (문장2): 드라마의 내용이 만점입니다. (답):모순간 후만 못합니다. (문장2): 이 드라마는 10점 만점을 넘는 점수를 주고 싶은 드라마입니다. (답):중립적인 프로그램을 진행하며 전시에 필요한 내용을 편성했습니다. (답):모순적으로 제작시간이 더 많은 편이었습니다. (답):중립될 계획입니다. (답):모순적으로 제작시간이 더 비싸지 않았습니다. (답):중립될 예정입니다. (답):중립될'

In [348]:
nli_train_dataset['test']

Dataset({
    features: ['guid', 'source', 'premise', 'hypothesis', 'label', 'input'],
    num_rows: 2500
})

In [349]:
nli_valid_dataset = nli_train_dataset['test'].map(
    tokenize, batched=True, remove_columns=['guid', 'source', 'premise', 'hypothesis', 'input']
)
nli_valid_dataset

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids'],
    num_rows: 2500
})

In [350]:
from torch.utils.data import DataLoader

batch_size=16
nli_val_ds = nli_valid_dataset
nli_val_ds.set_format(type='torch')
nli_val_dl = DataLoader(nli_val_ds, batch_size=batch_size)

In [351]:
import re
import torch
from tqdm import tqdm

def acc(pred,label):
    return torch.sum(torch.tensor(pred) == label.squeeze()).item()

In [352]:
model.eval()
val_losses = []
val_acc = 0

for step, batch in enumerate(tqdm(nli_val_dl)):
    label = batch['label']
    input_id= batch['input_ids'].to(device)

    pred = model.generate(input_id, max_length=150)
    decoded_pred = tokenizer.batch_decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    decoded_pred = [re.findall("\(답\):[함의|중립|모순]*", x)[0][4:6] if re.findall("\(답\):[함의|중립|모순]*", x) else 'none' for x in decoded_pred]
    decoded_pred = [label2int[x] if x in label2int else -1 for x in decoded_pred]
    val_acc += acc(decoded_pred, label)
    
print("val acc: ", val_acc/len(nli_val_dl.dataset))

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [01:37<00:00,  1.62it/s]

val acc:  1.0


In [353]:
val_acc

2500

In [354]:
len(nli_val_dl.dataset)

2500